# Iterator basics

* Usage 
    - Large dataset or memory-intensive operations 
    - returns an evaluation on-demand without closing the fucntion (Stateful lazy-evaluation)
    - `yeild` returns a value and `__next__()` calls for next value 
* A generator object is an iterator but not all iterators are generators 
    - A generaor function returns a generator object 
    - A generator function uses lazy-evaluation to yeild a sequence 

## Building a generator 

Following is a typical function that returns a list of $n$ dictionaries containing random values. 

In [22]:
def rand_dict(n):
    import random as r
    ret = []
    while n >= 0:
        ret.append({'value': round( r.random() ,3) } )
        n-=1
    return ret 

In [23]:
rand_dict(5)

[{'value': 0.715},
 {'value': 0.309},
 {'value': 0.597},
 {'value': 0.921},
 {'value': 0.497},
 {'value': 0.071}]

for huge dataset of intercommunicating threads data dont come as a single bulk, rather it is captured and exchanged parallarry over time. A generator function only evaluates the code when requested, called __lazy evaluation__. 

In [42]:
# a generator 
def rand_dict_gen(n):
    import random as r
    while n >= 0:
        # each 
        yield {'n':n,
               'value':round(r.random(),3)}
        n-=1

Assign the generator object as `x`, verify the type also

In [49]:
x=rand_dict_gen(5)

In [50]:
type(x)

generator

Now perform the lazy evaluation using the `__next__()`  method from the generator object with a 1 sec delay. each iteration would continue from where it left. The the call exceeds the number of items the generator object throws an __StopIteration__ exception, thus its handled by breakig the call loop. <br>
Also notice, once the generator object is completely traversed, you can't go back. In such a case you may need to re-initialize it.

In [51]:
import time as t
for i in range(10):
    try:
        print(x.__next__() ) #callss the lazy eval
        t.sleep(1)
    except:
        break

{'n': 5, 'value': 0.823}
{'n': 4, 'value': 0.581}
{'n': 3, 'value': 0.798}
{'n': 2, 'value': 0.002}
{'n': 1, 'value': 0.387}
{'n': 0, 'value': 0.947}


### Building a list from generator object

In [52]:
# option 1 : List typecast
list(rand_dict_gen(5))

[{'n': 5, 'value': 0.368},
 {'n': 4, 'value': 0.685},
 {'n': 3, 'value': 0.341},
 {'n': 2, 'value': 0.479},
 {'n': 1, 'value': 0.61},
 {'n': 0, 'value': 0.849}]

In [53]:
# option 2 : List comprehension
[i for i in rand_dict_gen(5)]

[{'n': 5, 'value': 0.934},
 {'n': 4, 'value': 0.953},
 {'n': 3, 'value': 0.271},
 {'n': 2, 'value': 0.054},
 {'n': 1, 'value': 0.978},
 {'n': 0, 'value': 0.422}]

In [58]:
# option 3 : conditional acceptance 
# get random dict if the random value >0.5
[i for i in rand_dict_gen(5) if i['value'] > 0.5]

[{'n': 4, 'value': 0.673},
 {'n': 3, 'value': 0.505},
 {'n': 2, 'value': 0.868},
 {'n': 1, 'value': 0.996}]

## Fibonacci Seq using generator 

In [69]:
def fibo(n):
    a,b= 0,1
    while n >= 0:
        c = a+b
        a = b
        b = c
        yield c
        n -= 1

In [71]:
n=5
[i for i in fibo(n)] 

[1, 2, 3, 5, 8, 13]

### Generator Pipeline
Connecting multiple generators in a cascases form. the follwoing example shows two genrator `gen_rand(m,c)` that generates a random integer $n\in[0,m]$ that is supplied to `fibo(n)` which generates first $n$ __even__ fibonacci numbers, $m \le c\in\mathbb{N}$. print the __fraction__ of even fibo numbers with respect to length of the random fibo sequence. 

In [96]:
def gen_rand(m,c):
    import random as r
    while c > 0:
        yield r.randint(1,m)
        c -= 1

In [112]:
import time 
c = 5  #count
m = 20 #max
print(f'printing {c} fibo seq(s) of max len = {m}')
for i in gen_rand(m,c):
    time.sleep(1)
    #fibo is defined before
    print('len=',i,' \t', 
          'frac=',round(
                      len( [j for j in fibo(i) if j%2==0] ) / i , 3 
                  )
         ) 

printing 5 fibo seq(s) of max len = 20
len= 7  	 frac= 0.429
len= 12  	 frac= 0.333
len= 20  	 frac= 0.35
len= 10  	 frac= 0.4
len= 17  	 frac= 0.353


# Generator as Contex manager
* Create all nessesary DS at the start of the scope
* removes all related garbage after the end of the scope
* Generators do this using __`try`__, __`yield`__ and __`finally`__ statement, implemented as 
    ```
    @contextmanager
    def gen_name([args]):
        try:
            #setup code
            yield
        finally:
            #wrap up code
    ```
* The following generator return each line of a given file on demand 

`testFile.csv` alrady exists in the local directory 

In [196]:
with open('testFile.csv','r') as fp:
    print(fp.read())

name , city , job 
Amar , Delhi , Manager
Bob  , London , Banker
Chen , Beijing , Developer




In [195]:
from contextlib import contextmanager
import time

@contextmanager
def line_from_file(filename):
    try:
        fp = open(filename,'r')
        for line in fp.readlines():
            yield line
    finally:
        fp.close()
        
with line_from_file('testFile.csv') as f:
    print(f)

name , city , job 



RuntimeError: generator didn't stop

# Coroutine 
* A generator produces values each time isumt's called, a Coroutine consumes. 
* the manipulation of supplied value to a coroutine is upto the code, and stops when reaches the `yield` statement 
* Technically a coroutine is built from a generator object but it is conceptually different. I can't iterate over sequences.  
* The difference with Function is, a coroutine is __Stateful__ or percistance. 
* `send()` sends a value to the coroutine

In [223]:
def simple_coroutine():
    count_s = 0 #string
    count_i = 0 #int
    count_o = 0 #other
    while True:
        try:
            x = yield  # recipient 
            if type(x) == str:
                count_s += 1
            elif type(x) == int:
                count_i += 1
            else:
                count_o += 1
            print(f'data={x} srings={count_s} ints={count_i} others={count_o}')
        except GeneratorExit:
            return -1

In [224]:
x = simple_coroutine() #create a coroutine object
x.__next__()           #initialize

In [225]:
import time
data = ['abc',123,145,12.2,'pqr']
for i in data:
    x.send(i)
    time.sleep(1)

data=abc srings=1 ints=0 others=0
data=123 srings=1 ints=1 others=0
data=145 srings=1 ints=2 others=0
data=12.2 srings=1 ints=2 others=1
data=pqr srings=2 ints=2 others=1


In [214]:
type(12.2) == int

False